In [0]:
import numpy as np
import pandas as pd

# Dataset

In [0]:
from google.colab import files
uploaded=files.upload()

Saving VDS_Gardiner VDS_NO.97_log.csv to VDS_Gardiner VDS_NO.97_log.csv


The Columns in Detail:

1st Column: Date/ time of the feed (Time Stamp)

2nd Column: Unique continuous ID for each detector base on the road direction (easy for subscription)

3rd Column: Detector Name in the system

4th Column: Detector physical address

5th Column: Detector's Latitude

6th Column: Detector's Longitude

7th Column: Integer represnting the lane at which vehicles are being detected (Lane Index)

8th Column: % of time loop is occupied per interval

9th Column: Average speed during an interval (km/h)

10th Column: vehicles per interval in (100 vehicle/hour)

11th Column: VdsDevice's unique ID

12th Colunn: Region Name where the detector is located

In [0]:
data = pd.read_csv("VDS_Gardiner VDS_NO.97_log.csv", header=None)
data[0:10]

0   1           2   ... 9     10         11
0  2012-06-25 00:00:00  97   de0060dwr  ...  0  1220   Gardiner
1  2012-06-25 00:02:00  97   de0060dwr  ...  0  1220   Gardiner
2  2012-06-25 00:03:00  97   de0060dwr  ...  0  1220   Gardiner
3  2012-06-25 00:05:00  97   de0060dwr  ...  0  1220   Gardiner
4  2012-06-25 00:06:00  97   de0060dwr  ...  0  1220   Gardiner
5  2012-06-25 00:07:00  97   de0060dwr  ...  0  1220   Gardiner
6  2012-06-25 00:09:00  97   de0060dwr  ...  0  1220   Gardiner
7  2012-06-25 00:10:00  97   de0060dwr  ...  0  1220   Gardiner
8  2012-06-25 00:12:00  97   de0060dwr  ...  0  1220   Gardiner
9  2012-06-25 00:13:00  97   de0060dwr  ...  1  1220   Gardiner

[10 rows x 12 columns]

#Neural Network

In [0]:
import torch
import torch.nn as nn
import torch.optim as optim

In [0]:
cuda = torch.device('cuda')

In [0]:
def get_model_name(name, batch_size, learning_rate, epoch, momentum):
 """ Generate a name for the model consisting of all the hyperparameter values
 Args:
 config: Configuration object containing the hyperparameters
 Returns:
 path: A string with the hyperparameter name and value concatenated
 """
  if momentum:
    path = "model_{0}_bs{1}_lr{2}_epoch{3}_momentum{4}".format(name,
            batch_size,
            learning_rate,
            epoch,momentum)
  else:
    path = "model_{0}_bs{1}_lr{2}_epoch{3}".format(name,
            batch_size,
            learning_rate,
            epoch)
  return path

In [0]:
class Model_1(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Model_1, self).__init__()
        self.name = "Model_1"
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        # Set an initial hidden state
        h0 = torch.zeros(1, input.size(0), self.hidden_size)
        # Forward propagate the RNN
        out, _ = self.rnn(input, h0)
        # Pass the output of the last time step to the classifier
        out = self.fc(torch.max(out, dim=1)[0])
        return out

In [0]:
def train_model(model, trainloader, validloader, batch_size = 32, learning_rate = 0.001, num_epochs = 30, momentum = 0.9):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)
    train_err = np.zeros(num_epochs)
    train_loss = np.zeros(num_epochs)
    val_err = np.zeros(num_epochs)
    val_loss = np.zeros(num_epochs)
    for epoch in range(num_epochs):
        for i, batch in enumerate(train_loader):
            print(i)
            input_data = batch.data[0]
            optimizer.zero_grad()
            outputs = model(input_data)
            labels = batch.label
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        train_err[epoch], train_loss[epoch] = get_accuracy(model, train_set, criterion, batch_size)
        val_err[epoch], val_loss[epoch] = get_accuracy(model, valid_set, criterion, batch_size)
        print(("Epoch {}: Train accuracy: {}, Train loss: {} |"+
               "Validation accuracy: {}, Validation loss: {}").format(
                   epoch + 1,
                   train_err[epoch],
                   train_loss[epoch],
                   val_err[epoch],
                   val_loss[epoch]))
        # Save the current model (checkpoint) to a file
        model_path = get_model_name(model.name, batch_size, learning_rate, epoch, momentum)
        torch.save(model.state_dict(), model_path)
    np.savetxt("{}_train_err.csv".format(model_path), train_err)  # FIXME  err should be accuracy
    np.savetxt("{}_train_loss.csv".format(model_path), train_loss)
    np.savetxt("{}_val_err.csv".format(model_path), val_err)
    np.savetxt("{}_val_loss.csv".format(model_path), val_loss)
    print('Finished Training')

